# 04 — Sensitivity Analysis

Sobol global sensitivity analysis (via SALib) and one-at-a-time (OAT) sweeps
to understand which parameters most strongly affect model outputs.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from SALib.sample import saltelli
from SALib.analyze import sobol

from market_abm import MarketModel, DEFAULT_PARAMS
from market_abm.analytics import compute_return_statistics, hill_estimator

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## Define problem for Sobol analysis

We analyze 5 key parameters: β, φ, χ, σ_noise, λ

In [ ]:
problem = {
    'num_vars': 5,
    'names': ['beta', 'phi', 'chi', 'noise_sigma', 'lambda_mm'],
    'bounds': [
        [0.1, 10.0],    # beta
        [0.1, 3.0],     # phi
        [0.1, 5.0],     # chi
        [0.01, 0.15],   # noise_sigma
        [0.001, 0.05],  # lambda_mm
    ],
}

# Generate Saltelli samples (N * (2D + 2) evaluations)
N_SAMPLES = 64  # Increase for more accuracy (e.g., 256)
param_values = saltelli.sample(problem, N_SAMPLES, calc_second_order=False)
print(f'Total model evaluations: {len(param_values)}')

In [ ]:
# Run simulations
Y_kurtosis = np.zeros(len(param_values))
Y_volatility = np.zeros(len(param_values))

for i, X in enumerate(param_values):
    params = {
        **DEFAULT_PARAMS,
        'steps': 1000,
        'n_agents': 100,
        'seed': 42 + i,
        'beta': X[0],
        'phi': X[1],
        'chi': X[2],
        'noise_sigma': X[3],
        'lambda_mm': X[4],
    }
    model = MarketModel(params)
    model.run()
    returns = np.array(model.market_maker.return_history)
    stats = compute_return_statistics(returns)
    Y_kurtosis[i] = stats['kurtosis']
    Y_volatility[i] = stats['std']
    
    if (i + 1) % 100 == 0:
        print(f'  {i+1}/{len(param_values)} done')

print('Sobol sampling complete.')

## Sobol indices

In [ ]:
Si_kurt = sobol.analyze(problem, Y_kurtosis, calc_second_order=False)
Si_vol = sobol.analyze(problem, Y_volatility, calc_second_order=False)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for ax, Si, title in [(axes[0], Si_kurt, 'Kurtosis'), (axes[1], Si_vol, 'Volatility')]:
    s1 = Si['S1']
    st = Si['ST']
    x = np.arange(len(problem['names']))
    width = 0.35
    ax.bar(x - width/2, s1, width, label='First-order (S1)', alpha=0.8)
    ax.bar(x + width/2, st, width, label='Total (ST)', alpha=0.8)
    ax.set_xticks(x)
    ax.set_xticklabels(problem['names'], rotation=30)
    ax.set_title(f'Sobol Indices — {title}')
    ax.set_ylabel('Sensitivity Index')
    ax.legend()
    ax.set_ylim(bottom=0)

plt.tight_layout()
fig.savefig('../figures/sobol_indices.png', dpi=150, bbox_inches='tight')
plt.show()

## OAT (One-at-a-time) Analysis

In [ ]:
oat_params = {
    'beta': np.linspace(0.1, 10.0, 15),
    'chi': np.linspace(0.1, 5.0, 15),
    'noise_sigma': np.linspace(0.01, 0.15, 15),
}

oat_results = {}

for pname, pvals in oat_params.items():
    kurtosis_vals = []
    for val in pvals:
        params = {**DEFAULT_PARAMS, 'steps': 2000, 'n_agents': 150, 'seed': 42}
        params[pname] = val
        model = MarketModel(params)
        model.run()
        returns = np.array(model.market_maker.return_history)
        kurtosis_vals.append(compute_return_statistics(returns)['kurtosis'])
    oat_results[pname] = (pvals, kurtosis_vals)

print('OAT analysis complete.')

In [ ]:
fig, axes = plt.subplots(1, len(oat_params), figsize=(5 * len(oat_params), 4))

for ax, (pname, (pvals, kvals)) in zip(axes, oat_results.items()):
    ax.plot(pvals, kvals, 'o-', markersize=4)
    ax.set_xlabel(pname)
    ax.set_ylabel('Excess Kurtosis')
    ax.set_title(f'Kurtosis vs {pname}')
    ax.axhline(0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
fig.savefig('../figures/oat_analysis.png', dpi=150, bbox_inches='tight')
plt.show()